In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import mplleaflet
import ipywidgets as w

### Import bike point, rail and tube station locations and stats

In [2]:
bps = pd.read_csv('data/bike-points.csv')
bps['Installed'] = pd.to_datetime(bps['Installed'])

all_stns = pd.read_csv('data/tube-lat-long-zone.csv')
all_tubes_ee = pd.read_csv('data/ee.csv')
all_og_stns = all_stns[~all_stns['Station'].isin(all_tubes_ee['Station'])]

In [3]:
max_zone = 3
tubes_ee = all_tubes_ee[all_tubes_ee['Zone']<=max_zone]
og_stns = all_og_stns[all_og_stns['Zone']<=max_zone]

og_lon = og_stns['Longitude']
og_lat = og_stns['Latitude']

In [4]:
phases = {'Start': pd.to_datetime('2010-07-30'),
        'April 2015': pd.to_datetime('2015-04-01'),        
        'Now': pd.to_datetime('now')}

- Service began late July 2010
- Service switched from Barclays to Santander sponsorship in April 2015

In [5]:
def draw_map():  
    sd = slider.value[0]
    ed = slider.value[1]
    start_date = phases[sd]
    end_date = phases[ed]
    ee_year = min(2017,end_date.year)
    
    bps_start = bps[bps['Installed']<=start_date]

    bps_start_lon = bps_start['lon']
    bps_start_lat = bps_start['lat']
    bps_start_docks = bps_start['NbDocks'] 
    
    bps_end = bps[bps['Installed']>start_date]
    bps_end = bps_end[bps_end['Installed']<end_date]

    bps_end_lon = bps_end['lon']
    bps_end_lat = bps_end['lat']
    bps_end_docks = bps_end['NbDocks'] 
    

    tubes_sel = tubes_ee[tubes_ee['Year']==ee_year]
    tubes_lon = tubes_sel['Longitude']
    tubes_lat = tubes_sel['Latitude']
    yr_ee = tubes_sel['Annual-e-and-e-million']  

    graph_ee = yr_ee * 3
    graph_docks_start = bps_start_docks * 3
    graph_docks_end = bps_end_docks * 3

    fig = plt.figure(figsize=(10,10)) 
    plt.scatter(bps_start_lon, bps_start_lat, marker = 'o', color = 'c',s=graph_docks_start, alpha = 0.5)
    plt.scatter(bps_end_lon, bps_end_lat, marker = 'o', color = 'r', s=graph_docks_end, alpha = 0.5)
    plt.scatter(tubes_lon, tubes_lat, s=graph_ee, marker = 'o', color = 'k',alpha = 0.5)
    plt.scatter(og_lon, og_lat, marker = 'o', color = 'y', alpha = 0.5)
    display(mplleaflet.display(fig=fig, tiles='cartodb_positron'))

### Map tube and bike point locations

- Black dots are located at tube stations and dot sizes show passenger volumes the year of the end time (last data 2017)
- Orange dots are located at other railway stations and are all the same size
- Blue dots show the bike points which were installed before the start time
- Red dots show the bike points which were installed after the start time and before the end time

In [6]:
def on_update(b):
    draw_map()

slider = w.SelectionRangeSlider(
    options= list(phases.keys()),
    value=('Start','Start'),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

butt = w.Button(
    description='Create map',
    disabled=False)

ctrls = w.VBox([slider,butt])
display(ctrls)
butt.on_click(on_update)

- Top map shows docking stations at the start of the service
- Bottom map shows docking stations installed prior to the April 2015 sponsorship change in blue, and docking stations installed since then in red

##### The maps will not be immediately visible via certain ways of viewing this notebook.
- If you are in an interactive environment, the 'top map' will be produced by clicking 'Create map' with the control at 'Start-Start', and the second with the control at 'April 2015' and 'Now'. Another option is to produce a HTML map in a separate window by running the cells below.

- Alternatively, view the notebook on nbviewer where the two maps will be visible but without the slider control.

In [1]:
def combo_map_html():  
    start_date = pd.to_datetime('2010-07-30')
    mid_date = pd.to_datetime('2015-04-01')
    end_date = pd.to_datetime('now')
    ee_year = 2017
    
    bps_start = bps[bps['Installed']<=start_date]

    bps_start_lon = bps_start['lon']
    bps_start_lat = bps_start['lat']
    bps_start_docks = bps_start['NbDocks'] 
    
    bps_mid = bps[(bps['Installed']<=mid_date) & (bps['Installed']>start_date)]

    bps_mid_lon = bps_mid['lon']
    bps_mid_lat = bps_mid['lat']
    bps_mid_docks = bps_mid['NbDocks'] 
    
    bps_end = bps[(bps['Installed']>mid_date) & (bps['Installed']<=end_date)]
    #bps_end = bps_end[bps_end['Installed']<=end_date]
    bps_end_lon = bps_end['lon']
    bps_end_lat = bps_end['lat']
    bps_end_docks = bps_end['NbDocks'] 
    

    tubes_sel = tubes_ee[tubes_ee['Year']==ee_year]
    tubes_lon = tubes_sel['Longitude']
    tubes_lat = tubes_sel['Latitude']
    yr_ee = tubes_sel['Annual-e-and-e-million']  

    graph_ee = yr_ee * 3
    graph_docks_start = bps_start_docks * 3
    graph_docks_mid = bps_mid_docks * 3
    graph_docks_end = bps_end_docks * 3

    fig = plt.figure(figsize=(10,10))
    
    plt.scatter(bps_start_lon, bps_start_lat, marker = 'o', color = 'c',s=graph_docks_start, alpha = 0.5)
    plt.scatter(bps_mid_lon, bps_mid_lat, marker = 'o', color = 'b',s=graph_docks_mid, alpha = 0.5)
    plt.scatter(bps_end_lon, bps_end_lat, marker = 'o', color = 'r', s=graph_docks_end, alpha = 0.5)
    plt.scatter(tubes_lon, tubes_lat, s=graph_ee, marker = 'o', color = 'k',alpha = 0.5)
    plt.scatter(og_lon, og_lat, marker = 'o', color = 'y', alpha = 0.5)
    mplleaflet.show(fig=fig, tiles='cartodb_positron')

In [8]:
combo_map_html()

### HTML map of tube and bike point locations

- Black dots are located at tube stations and dot sizes show passenger volumes in 2017
- Orange dots are located at other railway stations and are all the same size
- Cyan dots show the bike points which were installed at the start of the service (July 2010)
- Blue dots show the bike points which were installed after the original points and before the change of sponsorship
- Red dots show the bike points which were installed after the change of sponsorship (April 2015)